In [4]:
# Vitamin Supplements Minimization Problem
# Install required packages
!apt-get install -y -qq glpk-utils
!pip install pulp pandas

# Import required packages
import pulp as pp
import pandas as pd

# Initialize the MODEL (Minimization this time)
model = pp.LpProblem(name='vitamin-problem', sense=pp.LpMinimize)

# Declare the VARIABLES (two types of vitamin supplements)
SupplementA = pp.LpVariable(name="SupplementA", lowBound=0, cat='Continuous')
SupplementB = pp.LpVariable(name="SupplementB", lowBound=0, cat='Continuous')

# Cost coefficients (cost per unit of each supplement)
CostA = 0.05  # $0.05 per unit of Supplement A
CostB = 0.03  # $0.03 per unit of Supplement B

# Objective function (minimize total cost)
obj_func = CostA * SupplementA + CostB * SupplementB

# CONSTRAINTS (nutritional requirements)
# Vitamin X requirement: 2*A + 1*B >= 16
C1 = pp.LpConstraint(name='VitaminX',
                    e= 2*SupplementA + 1*SupplementB,
                    rhs=16,
                    sense=pp.LpConstraintGE)

# Vitamin Y requirement: 1*A + 1*B >= 12
C2 = pp.LpConstraint(name='VitaminY',
                    e= 1*SupplementA + 1*SupplementB,
                    rhs=12,
                    sense=pp.LpConstraintGE)

# Vitamin Z requirement: 1*A + 3*B >= 24
C3 = pp.LpConstraint(name='VitaminZ',
                    e= 1*SupplementA + 3*SupplementB,
                    rhs=24,
                    sense=pp.LpConstraintGE)

# Build MODEL
model += obj_func
model += C1
model += C2
model += C3

# Solve the MODEL
model.solve()

# Basic Report
print("Model Status:", pp.LpStatus[model.status])

# Get results
Results = {
    "Optimal Solution to minimize cost": pp.value(model.objective),
    "SupplementA": SupplementA.varValue,
    "SupplementB": SupplementB.varValue
}

# Display results
print("\nOptimal Solution:")
print(pd.DataFrame.from_dict(Results, orient='index', columns=['Value']).map('{:,.2f}'.format))

# Sensitivity Analysis
print("\nSensitivity Analysis:")
sensitivityValues = [{'constraints': name, 'shadow price': c.pi, 'slack': c.slack}
                    for name, c in model.constraints.items()]
print(pd.DataFrame(sensitivityValues).set_index('constraints').map('{:,.2f}'.format))

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Model Status: Optimal

Optimal Solution:
                                  Value
Optimal Solution to minimize cost  0.44
SupplementA                        4.00
SupplementB                        8.00

Sensitivity Analysis:
            shadow price  slack
constraints                    
VitaminX            0.02  -0.00
VitaminY            0.01  -0.00
VitaminZ            0.00  -4.00
